[![Open in
Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/slitvinov/gitlog/blob/main/poisson/9points.ipynb)

In [7]:
import itertools
from math import cos, sin, pi, sqrt
import numpy as np
import scipy
import statistics

def u(i, j):
    x = i / m * pi
    y = j / m * pi
    return cos(x**2) * sin(y)


def f(i, j):
    x = i / m * pi
    y = j / m * pi
    z = x * x
    s, c = sin(z), cos(z)
    return -((2 * s + 4 * z * c + c) * sin(y)) * pi**2


def boundary(i, j):
    cond = i == 0 or j == 0 or j == m or i == m
    return u(i, j) if cond else None


def add(i, j, c):
    val = boundary(i, j)
    if val is None:
        if (i, j) not in ik:
            ik[i, j] = len(ik)
        data.append(c)
        col.append(ik[i, j])
        row.append(len(rhs) - 1)
    else:
        rhs[-1] -= c * val

print("      m      error")
for m in 10, 20, 40, 80, 120, 160, 320:
    h = 1 / m
    ik = {}
    data = []
    col = []
    row = []
    rhs = []
    for i in range(m):
        for j in range(m):
            if boundary(i, j) is None:
              rhs.append(f(i, j) * h**2)
              add(i, j, -4)
              add(i - 1, j, 1)
              add(i + 1, j, 1)
              add(i, j - 1, 1)
              add(i, j + 1, 1)
    A = scipy.sparse.csr_matrix((data, (row, col)), dtype=float)
    sol = scipy.sparse.linalg.spsolve(A, rhs)
    err = []
    for s, t in zip(sol, ik):
        e = (s - u(*t))**2
        err.append(e)
    print(f"{m:7} {sqrt(statistics.mean(err)):10.2e}")

      m      error
     10   1.31e-01
     20   2.68e-02
     40   6.32e-03
     80   1.55e-03
    120   6.84e-04
    160   3.84e-04
    320   9.55e-05
